In [1]:
%reload_ext kedro.ipython

[10/02/25 22:23:36] INFO     Using                                                                  ]8;id=395619;file:///Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.12/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=258783;file:///Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.12/site-packages/kedro/framework/project/__init__.py#270\270]8;;\
                             '/Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3                
                             .12/site-packages/kedro/framework/project/rich_logging.yml' as logging                
                             configuration.                                                                        

                    INFO     Registered line magic '%reload_kedro'                                   ]8;id=705820;file:///Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=146810;file:///Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py#63\63]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=538532;file:///Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=419435;file:///Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py#65\65]8;;\

                    INFO     Resolved project path as:                                              ]8;id=504000;file:///Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=317129;file:///Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py#180\180]8;;\
                             /Users/eoinmolloy/Documents/Documents/FPL-Modelling/fpl-modelling.                    
                             To set a different path, run '%reload_kedro <project_root>'                           

[10/02/25 22:23:37] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=622152;file:///Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.12/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=840214;file:///Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.12/site-packages/kedro_telemetry/plugin.py#243\243]8;;\
                             the product. No personal data or IP addresses are stored on our side. To              
                             opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK` environment              
                             variables, or create a `.telemetry` file in the current working                       
                             directory with the contents `consent: false`. To hide this message,                   
                             explicitly grant or deny consent. Read more at                                        
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[10/02/25 22:23:38] INFO     Kedro project fpl_modelling                                            ]8;id=943654;file:///Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=373180;file:///Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py#146\146]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=262794;file:///Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=901775;file:///Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py#147\147]8;;\
                             'pipelines'                                                                           

In [2]:
current_squad = catalog.load("current_team")
players = catalog.load("players_merged")

                    INFO     Loading data from current_team (SQLQueryDataset)...               ]8;id=347660;file:///Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=6013;file:///Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py#1046\1046]8;;\

                    INFO     Loading data from players_merged (SQLQueryDataset)...             ]8;id=297187;file:///Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=292700;file:///Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py#1046\1046]8;;\

In [9]:
import pulp
import numpy as np
import pandas as pd
import logging

logger = logging.getLogger(__name__)


class BaseOptimizer:
    """Base class with shared functionality for FPL optimizers."""

    FORMATIONS = {
        "3-4-3": {"Defender": 3, "Midfielder": 4, "Forward": 3},
        "3-5-2": {"Defender": 3, "Midfielder": 5, "Forward": 2},
        "4-4-2": {"Defender": 4, "Midfielder": 4, "Forward": 2},
        "4-3-3": {"Defender": 4, "Midfielder": 3, "Forward": 3},
        "5-3-2": {"Defender": 5, "Midfielder": 3, "Forward": 2}
    }

    POSITION_REQUIREMENTS = {
        'Goalkeeper': 2,
        'Defender': 5,
        'Midfielder': 5,
        'Forward': 3
    }

    def __init__(self, df, kpi_col):
        self.df = df
        self.kpi_col = kpi_col
        self.all_players = df['player_name'].values
        self.n_players = len(df)

        self.player_info = {}
        for idx, row in df.iterrows():
            self.player_info[row['player_name']] = {
                'cost': row['now_cost'],
                'points': row[kpi_col],
                'position': row['position_name'],
                'team': row['team_id']
            }

    def validate_formation(self, formation):
        if formation not in self.FORMATIONS:
            print(f"Error: Unknown formation '{formation}'")
            print(f"Available formations: {list(self.FORMATIONS.keys())}")
            return False
        return True

    def add_squad_constraints(self, prob, x, team_size=15):
        prob += pulp.lpSum([x[p] for p in self.all_players]) == team_size

        for pos, required in self.POSITION_REQUIREMENTS.items():
            prob += pulp.lpSum([
                x[p] for p in self.all_players
                if self.player_info[p]['position'] == pos
            ]) == required

        all_teams = set(self.player_info[p]['team'] for p in self.all_players)
        for team in all_teams:
            prob += pulp.lpSum([
                x[p] for p in self.all_players
                if self.player_info[p]['team'] == team
            ]) <= 3

    def add_starting_xi_constraints(self, prob, x, s, formation_vars):
        """Flexible formation constraint (automatically chooses best formation)."""
        prob += pulp.lpSum([s[p] for p in self.all_players]) == 11

        for p in self.all_players:
            prob += s[p] <= x[p]

        prob += pulp.lpSum([
            s[p] for p in self.all_players
            if self.player_info[p]['position'] == 'Goalkeeper'
        ]) == 1

        prob += pulp.lpSum([formation_vars[f] for f in self.FORMATIONS.keys()]) == 1

        for pos in ['Defender', 'Midfielder', 'Forward']:
            prob += pulp.lpSum([
                s[p] for p in self.all_players
                if self.player_info[p]['position'] == pos
            ]) == pulp.lpSum([
                self.FORMATIONS[f][pos] * formation_vars[f]
                for f in self.FORMATIONS.keys()
            ])

    def add_captain_constraints(self, prob, s, c, v):
        prob += pulp.lpSum([c[p] for p in self.all_players]) == 1
        prob += pulp.lpSum([v[p] for p in self.all_players]) == 1

        for p in self.all_players:
            prob += c[p] <= s[p]
            prob += v[p] <= s[p]
            prob += c[p] + v[p] <= 1

    def create_objective(self, s, c):
        return pulp.lpSum([
            self.player_info[p]['points'] * s[p] for p in self.all_players
        ]) + pulp.lpSum([
            self.player_info[p]['points'] * c[p] for p in self.all_players
        ])

    def extract_solution(self, x, s, c, v, formation):
        squad = [p for p in self.all_players if x[p].value() == 1]
        starters = [p for p in self.all_players if s[p].value() == 1]
        bench = [p for p in squad if p not in starters]
        captain = [p for p in self.all_players if c[p].value() == 1][0]
        vice = [p for p in self.all_players if v[p].value() == 1][0]

        total_cost = sum(self.player_info[p]['cost'] for p in squad)
        expected_points = sum(self.player_info[p]['points'] for p in starters)
        expected_points += self.player_info[captain]['points']

        return {
            "squad": squad,
            "starters": starters,
            "bench": bench,
            "captain": captain,
            "vice_captain": vice,
            "formation": formation,
            "total_cost": total_cost,
            "expected_points": expected_points
        }

    def rank_squad(self, squad_players):
        squad_df = self.df[self.df['player_name'].isin(squad_players)].copy()
        squad_df = squad_df.sort_values(by=self.kpi_col, ascending=False)
        squad_df['rank'] = range(1, len(squad_df) + 1)
        return squad_df[['player_name', self.kpi_col, 'position_name', 'team_id', 'rank']]


class TeamOptimizer(BaseOptimizer):
    def solve(self, budget=100, debug=True):
        prob = pulp.LpProblem("FPL_Team_Selection", pulp.LpMaximize)

        x = pulp.LpVariable.dicts("squad", self.all_players, cat='Binary')
        s = pulp.LpVariable.dicts("start", self.all_players, cat='Binary')
        c = pulp.LpVariable.dicts("captain", self.all_players, cat='Binary')
        v = pulp.LpVariable.dicts("vice_captain", self.all_players, cat='Binary')
        formation_vars = pulp.LpVariable.dicts("formation", self.FORMATIONS.keys(), cat='Binary')

        prob += self.create_objective(s, c)
        prob += pulp.lpSum([
            self.player_info[p]['cost'] * x[p] for p in self.all_players
        ]) <= budget

        self.add_squad_constraints(prob, x)
        self.add_starting_xi_constraints(prob, x, s, formation_vars)
        self.add_captain_constraints(prob, s, c, v)

        prob.solve(pulp.PULP_CBC_CMD(msg=0))

        if pulp.LpStatus[prob.status] != "Optimal":
            print(f"❌ Solver status: {pulp.LpStatus[prob.status]}")
            return None

        chosen_formation = [f for f in self.FORMATIONS.keys() if formation_vars[f].value() == 1][0]
        result = self.extract_solution(x, s, c, v, chosen_formation)
        result['squad_ranking'] = self.rank_squad(result['squad'])
        self.print_team_solution(result)
        return result

    def print_team_solution(self, result):
        print("\n=== OPTIMAL TEAM ===")
        print(f"Total Cost: £{result['total_cost']}M")
        print(f"Expected Points: {result['expected_points']:.1f}")
        print(f"Formation: {result['formation']}")
        print(f"Captain: {result['captain']}")
        print(f"Vice Captain: {result['vice_captain']}")
        print("\nStarters:")
        for p in result['starters']:
            info = self.player_info[p]
            cap_marker = " (C)" if p == result['captain'] else " (VC)" if p == result['vice_captain'] else ""
            print(f"  {p} ({info['position']}){cap_marker} - £{info['cost']}M - {info['points']:.1f} pts")
        print("\nBench:")
        for p in result['bench']:
            info = self.player_info[p]
            print(f"  {p} ({info['position']}) - £{info['cost']}M - {info['points']:.1f} pts")


class TransferOptimizer(BaseOptimizer):
    def __init__(self, df, current_squad, kpi_col):
        super().__init__(df, kpi_col)
        self.current_squad = current_squad
        self.current_players = set(current_squad['player_name'].values)

        self.selling_prices = {}
        for idx, row in current_squad.iterrows():
            purchase_price = row.get('purchase_price', row['now_cost'])
            current_price = row['now_cost']
            if current_price > purchase_price:
                profit = current_price - purchase_price
                half_profit = int(profit * 10 / 2) / 10
                selling_price = purchase_price + half_profit
            else:
                selling_price = current_price
            self.selling_prices[row['player_name']] = selling_price

    def solve_transfers(self, 
                         budget=0,
                         free_transfers=1,
                         max_transfers=6,
                         players_to_remove=None,
                         players_to_keep=None,
                         points_penalty_per_transfer=4,
                         debug=True):

        players_to_remove = players_to_remove or []
        players_to_keep = players_to_keep or []

        players_to_remove = [p for p in players_to_remove if p in self.current_players]
        players_to_keep = [p for p in players_to_keep if p in self.current_players]

        if set(players_to_remove) & set(players_to_keep):
            print("Error: Players cannot be in both remove and keep lists")
            return None

        if debug:
            print(f"\n=== TRANSFER OPTIMIZATION ===")
            print(f"Current squad: {len(self.current_players)} players")
            print(f"Budget available: £{budget}M")
            print(f"Free transfers: {free_transfers}")
            print(f"Max transfers to consider: {max_transfers}")
            print(f"Points penalty per extra transfer: -{points_penalty_per_transfer}")
            if players_to_remove:
                print(f"Forced removals: {players_to_remove}")
            if players_to_keep:
                print(f"Protected players: {len(players_to_keep)}")

        prob = pulp.LpProblem("FPL_Transfer_Optimization", pulp.LpMaximize)

        x = pulp.LpVariable.dicts("in_squad", self.all_players, cat='Binary')
        transfer_out = pulp.LpVariable.dicts("transfer_out", self.all_players, cat='Binary')
        transfer_in = pulp.LpVariable.dicts("transfer_in", self.all_players, cat='Binary')
        s = pulp.LpVariable.dicts("start", self.all_players, cat='Binary')
        c = pulp.LpVariable.dicts("captain", self.all_players, cat='Binary')
        v = pulp.LpVariable.dicts("vice_captain", self.all_players, cat='Binary')
        formation_vars = pulp.LpVariable.dicts("formation", self.FORMATIONS.keys(), cat='Binary')

        num_transfers = pulp.LpVariable("num_transfers", lowBound=0, cat='Integer')
        extra_transfers = pulp.LpVariable("extra_transfers", lowBound=0, cat='Integer')

        points_from_team = self.create_objective(s, c)
        prob += points_from_team - (points_penalty_per_transfer * extra_transfers)

        for p in self.all_players:
            if p in self.current_players:
                prob += x[p] + transfer_out[p] == 1
                prob += transfer_in[p] == 0
            else:
                prob += x[p] == transfer_in[p]
                prob += transfer_out[p] == 0

        prob += num_transfers == pulp.lpSum([transfer_out[p] for p in self.all_players])
        prob += num_transfers <= max_transfers
        prob += extra_transfers >= num_transfers - free_transfers
        prob += extra_transfers >= 0
        prob += pulp.lpSum([transfer_out[p] for p in self.all_players]) == \
                pulp.lpSum([transfer_in[p] for p in self.all_players])

        for p in players_to_remove:
            prob += transfer_out[p] == 1
        for p in players_to_keep:
            prob += transfer_out[p] == 0

        transfer_cost = pulp.lpSum([
            self.player_info[p]['cost'] * transfer_in[p] for p in self.all_players
        ]) - pulp.lpSum([
            self.selling_prices.get(p, 0) * transfer_out[p] for p in self.all_players
        ])
        prob += transfer_cost <= budget

        self.add_squad_constraints(prob, x)
        self.add_starting_xi_constraints(prob, x, s, formation_vars)
        self.add_captain_constraints(prob, s, c, v)

        prob.solve(pulp.PULP_CBC_CMD(msg=0))

        if pulp.LpStatus[prob.status] != "Optimal":
            print(f"❌ Solver status: {pulp.LpStatus[prob.status]}")
            return None

        chosen_formation = [f for f in self.FORMATIONS.keys() if formation_vars[f].value() == 1][0]
        result = self.extract_solution(x, s, c, v, chosen_formation)
        result['transferred_out'] = [p for p in self.all_players if transfer_out[p].value() == 1]
        result['transferred_in'] = [p for p in self.all_players if transfer_in[p].value() == 1]
        result['num_transfers'] = int(num_transfers.value())
        result['free_transfers'] = free_transfers
        result['extra_transfers'] = int(extra_transfers.value()) if extra_transfers.value() else 0
        result['points_penalty'] = result['extra_transfers'] * points_penalty_per_transfer
        result['net_expected_points'] = result['expected_points'] - result['points_penalty']

        money_spent = sum(self.player_info[p]['cost'] for p in result['transferred_in'])
        money_gained = sum(self.selling_prices.get(p, 0) for p in result['transferred_out'])
        result['net_spent'] = money_spent - money_gained
        result['remaining_budget'] = budget - result['net_spent']
        result['squad_ranking'] = self.rank_squad(result['squad'])

        self.print_transfer_solution(result)
        return result

    def print_transfer_solution(self, result):
        print("\n=== TRANSFER PLAN ===")
        print(f"Formation chosen: {result['formation']}")
        print(f"Expected points: {result['net_expected_points']:.1f}")
        print(f"Free transfers: {result['free_transfers']}")
        print(f"Extra transfers: {result['extra_transfers']} (-{result['points_penalty']} pts penalty)")
        print(f"Net spent: £{result['net_spent']}M, Remaining budget: £{result['remaining_budget']}M")
        print("\nTransfers In:", result['transferred_in'])
        print("Transfers Out:", result['transferred_out'])
        print("\nSquad:")
        for p in result['squad']:
            info = self.player_info[p]
            print(f"  {p} ({info['position']}) - £{info['cost']}M - {info['points']:.1f} pts")


In [10]:
optimizer = TransferOptimizer(df=players,kpi_col='points_per_game', current_squad=current_squad)

In [12]:
optimizer.solve_transfers(budget=0,
                         free_transfers=5,
                         max_transfers=5,
                         players_to_remove=['Alisson Becker', 'Kai Havertz', 'Rayan Aït-Nouri'],
                         players_to_keep=None,
                         points_penalty_per_transfer=4,
                         debug=True)


=== TRANSFER OPTIMIZATION ===
Current squad: 15 players
Budget available: £0M
Free transfers: 5
Max transfers to consider: 5
Points penalty per extra transfer: -4
Forced removals: ['Alisson Becker', 'Kai Havertz', 'Rayan Aït-Nouri']

=== TRANSFER PLAN ===
Formation chosen: 4-3-3
Expected points: 74.0
Free transfers: 5
Extra transfers: 0 (-0 pts penalty)
Net spent: £0.0M, Remaining budget: £0.0M

Transfers In: ['Marc Guéhi', 'Erling Haaland', 'Wilson Isidor', 'Robin Roefs', 'Omar Alderete']
Transfers Out: ['Kai Havertz', 'Ollie Watkins', 'Alisson Becker', 'Jeremie Frimpong', 'Rayan Aït-Nouri']

Squad:
  William Saliba (Defender) - £6.0M - 3.8 pts
  Jurriën Timber (Defender) - £5.8M - 6.2 pts
  Morgan Rogers (Midfielder) - £6.9M - 2.8 pts
  Antoine Semenyo (Midfielder) - £7.8M - 8.0 pts
  Pedro Lomba Neto (Midfielder) - £7.0M - 3.0 pts
  Marc Guéhi (Defender) - £4.8M - 7.2 pts
  Jordan Pickford (Goalkeeper) - £5.5M - 4.3 pts
  Milos Kerkez (Defender) - £5.9M - 2.0 pts
  Erling Haaland (


{
    'squad': [
        'William Saliba',
        'Jurriën Timber',
        'Morgan Rogers',
        'Antoine Semenyo',
        'Pedro Lomba Neto',
        'Marc Guéhi',
        'Jordan Pickford',
        'Milos Kerkez',
        'Erling Haaland',
        'Anthony Elanga',
        'Wilson Isidor',
        'Robin Roefs',
        'Omar Alderete',
        'Brennan Johnson',
        'Jarrod Bowen'
    ],
    'starters': [
        'William Saliba',
        'Jurriën Timber',
        'Antoine Semenyo',
        'Pedro Lomba Neto',
        'Marc Guéhi',
        'Erling Haaland',
        'Wilson Isidor',
        'Robin Roefs',
        'Omar Alderete',
        'Brennan Johnson',
        'Jarrod Bowen'
    ],
    'bench': ['Morgan Rogers', 'Jordan Pickford', 'Milos Kerkez', 'Anthony Elanga'],
    'captain': 'Erling Haaland',
    'vice_captain': 'Antoine Semenyo',
    'formation': '4-3-3',
    'total_cost': 99.8,
    'expected_points': 74.0,
    'transferred_out': [
        'Kai Havertz',
        

In [ ]:
players[players['player_name']=='Rayan Aït-Nouri']

,player_name,total_points,total_minutes,now_cost,selected_by_percent,transfers_in,transfers_out,player_id,team_id,position_id,...,web_name,first_name,second_name,points_per_game,players_team,players_team_strength,players_team_strength_home,players_team_strength_away,position_name,position_name_abbr
446,Rayan Aït-Nouri,10,202,5.9,6.7,480274,2019149,402,13,2,...,Aït-Nouri,Rayan,Aït-Nouri,3.3,Man City,4,1275,1315,Defender,DEF


In [ ]:
import pandas as pd
import numpy as np

def create_random_squad(players_df, random_state=None):
    """
    Selects a valid 15-player FPL squad from the full players_df.
    
    players_df must include:
        - player_name
        - position_name
        - team_id
        - now_cost
    
    Returns:
        DataFrame of 15 players with added selling_price.
    """
    rng = np.random.default_rng(random_state)

    # Helper to sample players from a given position
    def sample_position(pos, n):
        return players_df[players_df["position_name"] == pos].sample(n, random_state=rng.integers(0, 1e6))

    # Sample by FPL squad rules
    gk = sample_position("Goalkeeper", 2)
    df = sample_position("Defender", 5)
    mid = sample_position("Midfielder", 5)
    fwd = sample_position("Forward", 3)

    squad = pd.concat([gk, df, mid, fwd]).reset_index(drop=True)

    # Cap at 3 per team_id (if violated, re-sample — simple loop for small test data)
    while (squad.groupby("team_id").size() > 3).any():
        gk = sample_position("Goalkeeper", 2)
        df = sample_position("Defender", 5)
        mid = sample_position("Midfielder", 5)
        fwd = sample_position("Forward", 3)
        squad = pd.concat([gk, df, mid, fwd]).reset_index(drop=True)

    # Add selling_price (randomly 0–0.2 below now_cost)
    squad["selling_price"] = squad["now_cost"] - rng.choice([0, 0.1, 0.2], size=len(squad))

    return squad


In [ ]:
catalog.load("current_team")

In [ ]:
# Wildcard
team_opt = TeamOptimizer(players, 'total_points')
result = team_opt.solve(budget=1e6)

# # # Weekly transfers
# opt = TransferOptimizer(
#     df=players,
#     kpi_col="points_per_game",
#     current_squad_df=current_squad_df,
#     bank_balance=1.5,    # £1.5m in bank
#     free_transfers=1     # 1 free transfer
# )



=== TEAM OPTIMIZER (FROM SCRATCH) ===
Budget: £1000000.0m
Formation: 4-3-3
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/5g/y4dx7m714l7ft2p__s3ybpn80000gn/T/6815b21c9df446cc86b45a7c46168d57-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/5g/y4dx7m714l7ft2p__s3ybpn80000gn/T/6815b21c9df446cc86b45a7c46168d57-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 3006 COLUMNS
At line 21645 RHS
At line 24647 BOUNDS
At line 27616 ENDATA
Problem MODEL has 3001 rows, 2968 columns and 11872 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 523 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 2226 strengthened rows, 0 substitutions
Cgl0004I processed model has 2258 rows, 2968 c

In [ ]:
result = opt.solve(formation="4-3-3", max_transfers=2, debug=True)


=== SOLVING TRANSFER OPTIMIZATION ===
Formation: 4-3-3
Max transfers: 2
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/eoinmolloy/Documents/Documents/FPL-Modelling/.venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/5g/y4dx7m714l7ft2p__s3ybpn80000gn/T/81a5dbda07014b18a69a604e50a69b78-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/5g/y4dx7m714l7ft2p__s3ybpn80000gn/T/81a5dbda07014b18a69a604e50a69b78-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 4493 COLUMNS
At line 31313 RHS
At line 35802 BOUNDS
At line 40256 ENDATA
Problem MODEL has 4488 rows, 4453 columns and 17082 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 47.8169 - 0.01 seconds
Cgl0002I 742 variables fixed
Cgl0003I 0 fixed, 1 tightened bounds, 2226 strengthened rows, 0 substitutions
Cgl0004I process